In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline

import scsavailability as scs
    
import statsmodels.api as sm
from scipy import stats

from scsavailability import features as feat, model as md, plotting as pt, score as sc


2020-12-22 11:56:20,840 - NumExpr defaulting to 8 threads.


In [2]:
path = 'C:/Users/Jamie.williams/OneDrive - Newton Europe Ltd/Castle Donnington/Data/'

In [3]:
at = pd.read_csv(path+'active_totes_20201210.csv')
av = pd.read_csv(path+'Availability_with_Grey&Blue_1811-0912.csv',names = ["timestamp","Pick Station","Availability","Blue Tote Loss","Grey Tote Loss"])
#fa = pd.read_csv(path + 'scs_tote_matched.csv')
scs_raw = pd.read_csv(path + 'Faults20_11-10_12.csv')

In [4]:
fa = feat.add_code(scs_raw)
fa, unmapped = feat.add_tote_colour(fa)

Running: Function "load_tote_lookup" (id=2241971445968) was called 1 times


In [5]:
at = feat.pre_process_AT(at)

Running: Function "pre_process_AT" (id=2241971448560) was called 1 times


In [6]:
av = feat.pre_process_av(av)

Running: Function "pre_process_av" (id=2241971449712) was called 1 times


In [7]:
fa = feat.preprocess_faults(fa,remove_same_location_faults = True)

Running: Function "preprocess_faults" (id=2241971447552) was called 1 times
Running: Function "load_module_lookup" (id=2241971449280) was called 1 times
Running: Function "load_ID_lookup" (id=2241971447984) was called 1 times
duplicated location faults removed - max duration kept
HOTFIX: Quadrant only faults, PTT Asset Code update


In [8]:
fa_floor = feat.floor_shift_time_fa(fa, shift=0)

Time shifted by 0Minutes


In [63]:
fa_sel = feat.get_data_faults(fa_floor, modules=[1],PTT = 'PTT011')

#fa_sel = feat.fault_select(fa_floor, fault_select_options = {'Tote Colour':['Both','Blue','Grey']})
                                                            
fa_agg = feat.faults_aggregate(fa_sel,fault_agg_level= 'Asset Code', agg_type = 'count')

Running: Function "load_PTT_lookup" (id=2241971447696) was called 3 times


In [68]:
av_sel,at_sel = feat.av_at_select(av, at, availability_select_options = {'Pick Station' : ['PTT011']}, remove_high_AT = True, AT_limit = None)

av_agg = feat.aggregate_availability(av_sel, agg_level = 'Module')
at_agg = feat.aggregate_totes(at_sel, agg_level = 'Module')

#at_agg = at_agg[at_agg['TOTES']>100]

Running: Function "aggregate_availability" (id=2241971447408) was called 7 times


In [69]:
df = feat.merge_av_fa_at(av_agg ,at_df=at_agg, fa_df = fa_agg , agg_level = 'Module')

In [70]:
X,y = md.gen_feat_var(df,target = 'Downtime', features = ['Totes','Faults'])
y=y

In [67]:
X_train, X_test, y_train, y_test = md.split(X,y,split_options = {'test_size': 0.3,
                                                                 'random_state': None})

RF_mdl,predictions_RF=md.run_RF_model(X_train, X_test, y_train, y_test,RF_options = {'num_trees': 100, 
                                                                                    'criterion': 'mse', 
                                                                                    'max_depth': None, 
                                                                                    'dtree': False})

Linear_mdl,predictions_LM,Coeff,fit_metrics = md.run_LR_model(X_train, X_test, y_train, y_test)

XGB_mdl,predictions_XGB,Importance_XGB = md.run_XGB_model(X_train, X_test, y_train, y_test, XGB_options = {'num_trees': 100,
                                                                                                         'max_depth': None})

selection_options= {'threshold': -np.inf,
                    'max_features': None}

X_sel_rf = md.select_features(X, y, RF_mdl, **selection_options)

X_sel_lm = md.select_features(X, y, Linear_mdl, **selection_options)

#X_sel_XGB = md.select_features(X, y, XGB_mdl, selection_options)

X_train, X_test, y_train, y_test = md.split(X_sel_lm,y,split_options = {'test_size': 0.3,
                                                                       'random_state': None})

Linear_mdl,predictions_LM,Coeff,fit_metrics = md.run_LR_model(X_train, X_test, y_train, y_test)

cv_R2 = md.cross_validate_r2(Linear_mdl, X, y)

cv_R2 = md.cross_validate_r2(RF_mdl,X, y)

cv_R2 = md.cross_validate_r2(XGB_mdl, X, y)

Feature Importance Ranking: 
 
    Importance      Feature
0    0.481614        TOTES
1    0.031564  C1603BCL054
2    0.025440  C1603BCL014
3    0.024484  C2302STA014
4    0.020553  C2302RDC015
5    0.020262       SCS011
6    0.018990  C1603STA253
7    0.016609    C05PTT011
8    0.016364  C2301STA175
9    0.015161  C2301BCL177

Regression Metrics: 
 
           RF Metrics
MAE         0.119038
MSE         0.021257
RMSE        0.145799
MAPE%            inf
ACC%            -inf
OOB         0.366939
R2_Train    0.916159
R2_Pred     0.399319 

Feature Importance Ranking: 
 
    Importance      Feature
0    0.104421        TOTES
1    0.043242  C1502ACH004
2    0.041346  C2303STA222
3    0.037173  C0503RDC254
4    0.035301  C2302ACH009
5    0.034451  C1502STA246
6    0.030390  C1501RDC232
7    0.028040  C1603BCL097
8    0.027885  C0502STA026
9    0.022625       SCS014

Regression Metrics: 
 
          XGB Metrics
MAE         0.120813
MSE        0.0223437
RMSE        0.149478
MAPE%            

### Add LM - metrics

In [ ]:
sc.lm_coefficients(Linear_mdl, X_test ,y_test)

In [ ]:
X_train = sm.add_constant(X_train)
model = sm.OLS(y_train,X_train)
results = model.fit()
print(results.summary())

X_new = sm.add_constant(X_test)
#y_prediction =  results.predict(X_test)

# plt.scatter(y_prediction,y_test)

# from sklearn.metrics import r2_score
# print(r2_score(y_prediction,y_test))

In [ ]:
fa_Both = fa[fa['Tote Colour']=="Both"]
fa_Both = fa_Both[fa_Both['Pick Station']==False]
fa_Both[fa_Both['Desk']!="Z"]['Asset Code'].value_counts()

In [ ]:
faPT = fa[fa['Pick Station']!=False]
faPT[faPT['PLC']=='C10']

In [ ]:
fa_sel[fa_sel['Pick Station']!=False]['Asset Code'].value_counts()

In [71]:
X

,C05,C0501ACH011,C0501RDC004,C0501STA002,C0501STA012,C0501STA020,C0501STA201,C0501STA217,C0501STA219,C0501STA228,...,C2303STA222,SCS011,SCS012,SCS013,SCS014,SCS015,SCS016,SCSM01,Module,TOTES
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,1,36.271186
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,2,38.745763
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,3,83.648148
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,4,61.052632
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,5,59.152542
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8386,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,113,0,0,35,15,26.843137
8387,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,113,0,0,35,17,41.533333
8388,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,113,0,0,35,18,73.516667
8389,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,113,0,0,35,19,56.033333


In [72]:
y

0       0.183832
1       0.183832
2       0.183832
3       0.183832
4       0.183832
          ...   
8386    0.401005
8387    0.401005
8388    0.401005
8389    0.401005
8390    0.401005
Name: Downtime, Length: 8391, dtype: float64

In [61]:
at_agg.loc["2020-11-20":].head(50)

,Module,TOTES
timestamp,,
2020-11-20 00:00:00,1,36.271186
2020-11-20 00:00:00,2,38.745763
2020-11-20 00:00:00,3,83.648148
2020-11-20 00:00:00,4,61.052632
2020-11-20 00:00:00,5,59.152542
2020-11-20 00:00:00,7,42.864407
2020-11-20 00:00:00,8,52.945455
2020-11-20 00:00:00,9,45.474576
2020-11-20 00:00:00,10,50.762712
